In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os

In [2]:
# Need a conssitent (and small) size for the xraeys so we can process them with a CNN
# 

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),  # for compatibility
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # since we are going to use resnet18 as a pretrained model, we ned to map the greyscale to RGB with the same mean and standard deiation for RGB
])

In [3]:
# NExt lets load in the dataset

data_dir = "chest_xray"

## PyTorch has a built in data loader for image datasets with a particular directory structure
## It will automatically assign labels to to the classes in the folder and transform the images according to transform defined above

train_set = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=transform)
val_set = datasets.ImageFolder(os.path.join(data_dir, "val"), transform=transform)
test_set = datasets.ImageFolder(os.path.join(data_dir, "test"), transform=transform)

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32)
test_loader = DataLoader(test_set, batch_size=32)

In [4]:
# We will load in resnet18 as our base model and uyse reansfer learning to fine tune on our dataset.

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # I'm running a GTX1060

model = models.resnet18(pretrained=True) # PyTorch has resnet18 built in, making this extremely easy to do
model.fc = nn.Linear(model.fc.in_features, 2)  # binary classification
model = model.to(device)

C:\Users\liam_\anaconda3\envs\mlenv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\liam_\anaconda3\envs\mlenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
n_epochs = 2
def train_model(num_epochs=1):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")
train_model(n_epochs)

Epoch 1/2, Loss: 0.0045
Epoch 2/2, Loss: 0.0113


In [11]:
# OK great, the model is trained.

_=model.eval() # Let's switch to evaluation mode
correct = 0
total = 0

model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f"Accuracy: {accuracy:.2%}")

Accuracy: 83.65%


In [12]:
import pandas as pd
from torch.nn.functional import softmax

results = []


# Turn off gradients and run on the test set
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        probs = softmax(outputs, dim=1) # convert the model outputs from relative confidence to probability
        preds = torch.argmax(probs, dim=1)

        for i in range(images.size(0)):
            filename = test_loader.dataset.samples[i][0].split("/")[-1]
            true_label = labels[i].item()
            predicted_label = preds[i].item()
            confidence = probs[i][predicted_label].item()

            results.append({
                "filename": filename,
                "true_label": true_label,
                "predicted_label": predicted_label,
                "confidence": confidence
            })

df = pd.DataFrame(results)



# Let's also add in some simulated meta data so we have more to work with in PowerBI
import random

def generate_fake_data(df):
    df["age"] = [random.randint(20, 85) for _ in range(len(df))]
    df["gender"] = [random.choice(["M", "F"]) for _ in range(len(df))]
    df["hospital"] = [random.choice(["General", "CityMed", "HealthPlus"]) for _ in range(len(df))]
    return df

df = generate_fake_data(df)
df.to_csv(f"xray_predictions_with_meta_resnet18_basic_{n_epochs}epochs.csv", index=False)

In [9]:
df

,filename,true_label,predicted_label,confidence,age,gender,hospital
0,chest_xray\test\NORMAL\IM-0001-0001.jpeg,0,0,0.849889,33,M,CityMed
1,chest_xray\test\NORMAL\IM-0003-0001.jpeg,0,1,0.966549,56,F,HealthPlus
2,chest_xray\test\NORMAL\IM-0005-0001.jpeg,0,0,0.965710,35,M,HealthPlus
3,chest_xray\test\NORMAL\IM-0006-0001.jpeg,0,1,0.749216,80,M,General
4,chest_xray\test\NORMAL\IM-0007-0001.jpeg,0,0,0.992004,24,F,HealthPlus
...,...,...,...,...,...,...,...
619,chest_xray\test\NORMAL\IM-0015-0001.jpeg,1,1,0.999978,32,F,CityMed
620,chest_xray\test\NORMAL\IM-0016-0001.jpeg,1,1,0.999997,22,F,CityMed
621,chest_xray\test\NORMAL\IM-0017-0001.jpeg,1,1,0.999864,53,M,General
622,chest_xray\test\NORMAL\IM-0019-0001.jpeg,1,1,0.999997,46,F,General
